# Lending Club Case Study

In [98]:
#import the libraries
import pandas as pd #To work with dataset
import numpy as np #Math library
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
import plotly.tools as tls # It's useful to we get some tools of plotly
import warnings # This library will be used to ignore some warnings
from collections import Counter # To do counter of some features
import plotly.io as pio

In [99]:
#function definitions 
ENABLE_DBG_LOG = True
ENABLE_INFO_LOG = True
NA_CUT_OFF_PERC = 10 #upto this %, nulls are allowed  


#retrieve cols from the df which are to be used for analysis. 
#to_be_used columns is used for picking them
def filter_cols(df):
    df = df [df['to_be_used'] == 1] 
    srs_cols = df['LoanStatNew']
    return srs_cols

def log_debug(msg,data):
    if ENABLE_DBG_LOG:
        print(msg,data)

def log_info(msg,data):
    if ENABLE_INFO_LOG:
        print(msg,data)
    
def select_cols_to_analyze(df,df_col_defs):
    #pick cols that are picked by the definition of the column 
    #(this does not consider the sanity of values at this point)
    srs_cols = filter_cols(df_col_defs)
    # log_debug('Cols picked for analysis \n', srs_cols)
    #analyze the data in these columns to determine whether to consider this column
    #if more data is empty or null, don't bother
    total_rows = len(df)
    log_info("Total rows in df ", total_rows)
    final_cols_lst = []
    for index, col in srs_cols.items():
        #check the data in col in data df 
        if col in df:
            na_count = df[col].isna().sum()
            na_perc = ( na_count / total_rows) * 100;
            if na_perc < NA_CUT_OFF_PERC:
                final_cols_lst.append(col)
            else:
                log_debug("col " + col + " is rejected at high na perc",na_perc )
        else:
            log_debug("col " + col + " is rejected as not present",'' )
    return final_cols_lst

def render_segmented_univariate_analysis(df,col,segments,title):
    bar_data = []
    for segment in segments:
        segment_bar = go.Bar(
                    x = df_data[df_data[col] == segment][col].value_counts().index.values,
                    y = df_data[df_data[col] == segment][col].value_counts().values,
                    name=segment
                    )
        bar_data.append(segment_bar)
    
    layout = go.Layout(
    )

    layout = go.Layout(
        yaxis=dict(
            title='Count'
        ),
        xaxis=dict(
            title=title
        ),
        title=title + ' distribution'
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='grouped-bar')


#Pivot_col indicates the column to pivot. For e.g Loan status could be pivot and 
#home_ownership status as variant to find the distribution 
def render_bivariate_analysis(df,pivot_col,col2):
    pivot_col_values = df[pivot_col].unique()
    for pcol_val in pivot_col_values:
        df_tmp = df.loc[df[pivot_col] == pcol_val][col2].values.tolist()
        hist = go.Histogram(
            x=df_tmp,
            histnorm='probability',
            name=pcol_val + ' Distribution'
        )


In [100]:
df = pd.read_csv('loan.csv')
df_col_defs = pd.read_csv('LoanStats_Selections.csv')

#pick cols to analyze
cols_to_analyze = select_cols_to_analyze(df,df_col_defs)
log_info('cols_to_analyze ',cols_to_analyze)

df_data = df[cols_to_analyze]
#filter out the current status values. 
df_data = df_data[~(df_data['loan_status'] == 'Current')]

#plot the bar chart to know hows the Fully Paid vs Charged Off distribution 
render_segmented_univariate_analysis(df_data,'loan_status', ['Charged Off','Fully Paid'],'Payment Status')

print(df_data['purpose'].value_counts())

# df[pivot_col].unique()

#plot 

#when an applicant submits the application, below are the items checked 
#Whether the user has history with LC
#which age group user belongs to 
#what is the asked amount and what is the occupatio / income 
#

# print(df['acc_open_past_24mths'])
#suspected columns that can be used 
# acc_now_delinq, acc_open_past_24mths, all_util, annual_inc, application_type, avg_cur_bal, chargeoff_within_12_mths,
# collection_recovery_fee, delinq_2yrs, delinq_amnt, dti, 

#there can be joint and well individual accounts 
#application_type, annual_inc_joint, 
#avg_cur_bal => what will be the value for joint types, chargeoff_within_12_mths, dti_joint

#didnt understand these fields 
# bc_open_to_buy, bc_util, collections_12_mths_ex_med, earliest_cr_line, 

#check if any useful info is present in these fields 
#emp_length, emp_title 

# print("Columns ",df.columns)
# df.head()
# print(df.info())
# print(df.describe())

# columns = ["loan_amnt", "funded_amnt", "funded_amnt_inv", "term", "int_rate", "installment", "grade", "sub_grade", "emp_title", "emp_length", "home_ownership", "verification_status", "loan_status", "purpose", "addr_state", "dti","delinq_2yrs","inq_last_6mths",]
# 



Total rows in df  39717
col acc_open_past_24mths is rejected at high na perc 100.0
col annual_inc_joint is rejected at high na perc 100.0
col avg_cur_bal is rejected at high na perc 100.0
col bc_open_to_buy is rejected at high na perc 100.0
col bc_util is rejected at high na perc 100.0
col dti_joint is rejected at high na perc 100.0
col fico_range_high is rejected as not present 
col fico_range_low is rejected as not present 
col mort_acc is rejected at high na perc 100.0
col mths_since_last_delinq is rejected at high na perc 64.66248709620565
col mths_since_last_major_derog is rejected at high na perc 100.0
col tot_cur_bal is rejected at high na perc 100.0
cols_to_analyze  ['acc_now_delinq', 'addr_state', 'annual_inc', 'application_type', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'dti', 'emp_length', 'emp_title', 'funded_amnt', 'funded_amnt_inv', 'grade', 'home_ownership', 'loan_amnt', 'loan_status', 'purpose', '

debt_consolidation    18055
credit_card            5027
other                  3865
home_improvement       2875
major_purchase         2150
small_business         1754
car                    1499
wedding                 926
medical                 681
moving                  576
vacation                375
house                   367
educational             325
renewable_energy        102
Name: purpose, dtype: int64


In [31]:
# df.corr()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,1.000000,0.993650,0.141919,0.152286,0.249547,0.086587,0.008731,0.095983,-0.008644,-0.042378,...,NaN,NaN,NaN,NaN,-0.009505,NaN,NaN,NaN,NaN,NaN
member_id,0.993650,1.000000,0.140710,0.150322,0.257887,0.081025,0.009380,0.096963,-0.008119,-0.047086,...,NaN,NaN,NaN,NaN,-0.008769,NaN,NaN,NaN,NaN,NaN
loan_amnt,0.141919,0.140710,1.000000,0.981578,0.940034,0.930288,0.271149,0.066439,-0.031864,0.009229,...,NaN,NaN,NaN,NaN,-0.037180,NaN,NaN,NaN,NaN,NaN
funded_amnt,0.152286,0.150322,0.981578,1.000000,0.958422,0.956159,0.266965,0.066283,-0.032355,0.009259,...,NaN,NaN,NaN,NaN,-0.038502,NaN,NaN,NaN,NaN,NaN
funded_amnt_inv,0.249547,0.257887,0.940034,0.958422,1.000000,0.905039,0.254375,0.074689,-0.038501,-0.005712,...,NaN,NaN,NaN,NaN,-0.042746,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tax_liens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tot_hi_cred_lim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_bal_ex_mort,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_bc_limit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
